In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
from datetime import datetime as dt
from typing import *
import rfpy
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from rfpy.main import export_metadata
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import *
import numpy as np
import pandas as pd

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 

Caminho das pastas de entrada e saída

In [ ]:
entrada = Path(r'C:\Users\rsilva\Downloads\entrada')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

Função que mapeia os arquivos da pasta e retorna um lista generalizada `L`

In [ ]:
arquivos = get_files(entrada) ;  arquivos

Peguemos 1 arquivo com somente 1 thread_id por bloco e 1 com múltiplos thread_ids por bloco.

In [ ]:
one_tid = arquivos[-1]
mult_tid = arquivos[1]

On a normal conda environment the time of the following cell was
```python
%%timeit
one = process_bin(one_tid)
```
1.22 s ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%time
one = parse_bin(one_tid)

On a normal conda environment the time of the following cell was
```python
%%timeit
multi = process_bin(mult_tid)
```
1.29 s ± 73.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%time
multi = parse_bin(mult_tid)

In [ ]:
multi.keys()

In [ ]:
multi['file_version']

In [ ]:
multi['string']

In [ ]:
multi['blocos']

In [ ]:
exemplo = multi['blocos'][(21,0)][0]

In [ ]:
bytes_map = exemplo[0]
bloco = exemplo[1]

In [ ]:
bin_data = bloco.data ; bin_data

In [ ]:
len(bin_data)

In [ ]:
getattrs(bloco)

### Mapeamento dos bytes do arquivo nos blocos

Vamos testar se o mapeamento do bloco nos bytes do arquivo está correto, para tal vamos ler o arquivo novamente.

In [ ]:
with open(mult_tid, mode='rb') as bfile:
    # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
    body = bfile.read()

Primeiramente criamos um Bloco Base - Cabeçalho, Dados e Rodapé com a função `create_base_block`.

In [ ]:
show_doc(create_base_block)

In [ ]:
block = create_base_block(body[slice(*bytes_map)])

Como vemos na documentação da função acima ela aceita arquivos abertos lidos como `bytes`. Passamos para ela o arquivo aberto `body`, mas somente selecionando deste os índices de bytes `bytes_map` retornados acima.
`body[slice(*bytes_map)]` é o mesmo que `body[bytes_map[0]:bytes_map[1]]` mas numa notação mais _pythônica_   

In [ ]:
block.data

A função de teste a seguir testa a igualdade do bloco de dados acima e do bloco de dados diretamente mapeado nos bins, o teste foi bem sucedido caso nada seja retornado

In [ ]:
test_eq(bin_data, block.data)

In [ ]:
%%time
metadados = extract_metadata(multi)

In [ ]:
metadados

In [ ]:
export_metadata(entrada, saida)